# SimBOL's GRO Plugin: SBOL3 to GRO Simulation File Converter

Welcome to SimBOL, a tool to convert your genetic circuit designs from SBOL3 to `.gro` files for simulation.

**Workflow Overview:**
1.  **Initial Setup:** We'll install necessary dependencies and clone the SimBOL repository.
2.  **Upload SBOL3 File:** You will upload your design file in SBOL3 format.
3.  **(Automatic) JSON Processing:** Your SBOL3 file will be converted into an internal JSON format.
4.  **Simulation Parameter Configuration:** A graphical user interface will appear for you to adjust parameters for the GRO simulation.
5.  **Generate and Download `.gro` File:** Your `.gro` file will be generated, and you'll be able to download it.

Let's get started! Please run the cells in order.

In [ ]:
#@title 1. Environment Setup & Dependency Installation

print("Installing dependencies...")
!pip install sbol3 xmltodict lxml jsonschema pandas matplotlib seaborn ipywidgets gitpython --quiet
print("Dependencies installed.")

print("\nCloning the SimBOL repository...")
# Remove the repository if it already exists to ensure a fresh clone
!rm -rf SimBOL
!git clone https://github.com/frnvnd4/SimBOL.git
print("Repository cloned successfully.")

import sys
sys.path.append('/content/SimBOL')
print("Project path added to system.")

# General imports
import json
from google.colab import files
from IPython.display import display, clear_output, HTML

# Specific imports from the SimBOL project modules
try:
    from sbol3_to_json_converter.process_sbol import process_sbol
    from sbol3_to_json_converter.upload_sbol_file import upload_sbol_file
    from json_to_gro_generator.ui_parameters import display_form
    from json_to_gro_generator.params import prepare_parameters_and_data
    from json_to_gro_generator.gro_file_compiler import generate_gro_file
    print("SimBOL project modules loaded successfully.")
except ImportError as e:
    print(f"Error importing project modules: {e}")
    print("Please ensure the file structure in GitHub and __init__.py files are correct.")

display(HTML("<hr><b><font color='green'>Setup complete</font></b> You can proceed to the next step."))


Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 542.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
Dependencies installed.

Cloning the SimBOL repository...
Cloning into 'SimBOL'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 59 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 45.13 KiB | 1004.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.
Repository cloned successfully.
Project path added to system.
SimBOL project modules loaded successfully.


In [ ]:
#@title 2. Upload SBOL3 File

sbol_file_path = None
sbol_file_name_only = None

try:
    sbol_file_path, sbol_file_name_only = upload_sbol_file()
    if sbol_file_path:
        display(HTML(f"<hr><b><font color='green'>File '{sbol_file_name_only}' uploaded and saved as '{sbol_file_path}'</font></b>"))
    else:
        display(HTML(f"<hr><b><font color='red'>File upload cancelled or failed.</font></b>"))
except Exception as e:
    print(f"An error occurred during file upload: {e}")
    display(HTML(f"<hr><b><font color='red'>Error during file upload.</font></b>"))

Please select your SBOL 3 file to upload:



Saving BBa_I5200.xml to BBa_I5200.xml
------- File uploaded successfully: BBa_I5200.xml -------


In [ ]:
#@title 3. Convert SBOL3 to JSON

sbol_data_json_internal = None

if 'sbol_file_path' in locals() and sbol_file_path and 'sbol_file_name_only' in locals() and sbol_file_name_only:
    try:
        print(f"Processing '{sbol_file_name_only}'...")
        output_clean_json_path = process_sbol(sbol_file_path, sbol_file_name_only)

        if output_clean_json_path:
            print(f"SBOL file processed. Summarized JSON generated: {output_clean_json_path}")
            # Load the JSON for use in subsequent steps
            with open(output_clean_json_path, "r") as f:
                sbol_data_json_internal = json.load(f)
            display(HTML(f"<hr><b><font color='green'>SBOL file processed. Summarized JSON generated and loaded.</font></b>"))
        else:
            print("The process_sbol function did not return a valid JSON file path.")
            display(HTML(f"<hr><b><font color='red'>Error generating the JSON file.</font></b>"))

    except Exception as e:
        print(f"An error occurred during SBOL file processing: {e}")
        display(HTML(f"<hr><b><font color='red'>Error processing the SBOL file.</font></b>"))
else:
    print("No SBOL3 file was uploaded in the previous step. Please run the upload cell first.")
    display(HTML(f"<hr><b><font color='orange'>Warning: No SBOL file found to process.</font></b>"))

Processing 'BBa_I5200'...
Processing SBOL file...
JSON file generated successfully: BBa_I5200_clean.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SBOL file processed. Summarized JSON generated: BBa_I5200_clean.json


In [ ]:
#@title 4. Display GRO Parameter Form

gro_simulation_parameters = None
extracted_genes_data = None
extracted_qs_actions = None
extracted_biochemical_reactions = None

if 'sbol_data_json_internal' in locals() and sbol_data_json_internal:
    try:
        gro_simulation_parameters, extracted_genes_data, extracted_qs_actions, extracted_biochemical_reactions = display_form(sbol_data_json_internal)
        # Save confirmation is handled within display_form
    except Exception as e:
        print(f"An error occurred while displaying the form: {e}")
        display(HTML(f"<hr><b><font color='red'>Error displaying the parameter form.</font></b>"))
else:
    print("SBOL JSON data (sbol_data_json_internal) has not been loaded.")
    print("Please ensure you have successfully run the previous steps (SBOL Upload and Processing).")
    display(HTML(f"<hr><b><font color='orange'>Warning: No data available for the form. Execute previous steps.</font></b>"))

In [ ]:
#@title 5. Generate and Download .gro File

if 'gro_simulation_parameters' in locals() and gro_simulation_parameters:
    if extracted_genes_data is not None and extracted_qs_actions is not None and extracted_biochemical_reactions is not None:
        try:
            print("Preparing final data for .gro file generation...")
            # Use sbol_data_json_internal which is the loaded JSON
            signals, protein_actions, degradation_actions = prepare_parameters_and_data(sbol_data_json_internal, gro_simulation_parameters)

            # Determine the output file name
            base_name_for_output = "simulation_output"
            if 'sbol_file_name_only' in locals() and sbol_file_name_only:
                base_name_for_output = sbol_file_name_only.split('.')[0]

            gro_output_file_name = f"{base_name_for_output}.gro"

            print(f"Generating file '{gro_output_file_name}'...")
            generate_gro_file(gro_simulation_parameters, extracted_genes_data, extracted_qs_actions, signals, protein_actions, degradation_actions, extracted_biochemical_reactions, gro_output_file_name)
            print(f"File '{gro_output_file_name}' generated successfully.")

            files.download(gro_output_file_name)
            display(HTML(f"<hr><b><font color='green'>.gro file generated and download initiated</font></b>"))

        except Exception as e:
            print(f"An error occurred during .gro file generation or download: {e}")
            display(HTML(f"<hr><b><font color='red'>Error generating/downloading the .gro file.</font></b>"))
    else:
        print("Some extracted data (genes, QS, reactions) is missing. Was the form completed correctly?")
        display(HTML(f"<hr><b><font color='orange'>Warning: Missing data to generate .gro file.</font></b>"))
else:
    print("Simulation parameters (gro_simulation_parameters) have not been saved.")
    print("Please ensure you have interacted with the form and pressed 'Save Parameters'.")
    display(HTML(f"<hr><b><font color='orange'>Warning: No parameters available to generate .gro file.</font></b>"))

Preparing final data for .gro file generation...
Generating file 'BBa_I5200.gro'...
File 'BBa_I5200.gro' generated successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>